In [ ]:
import re
import MySQLdb
import bz2

def get_errors(cid, fullLog):
    try:
        socket = MySQLdb.connect("148.60.11.195", "script", "irmadb2019", "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT errlog_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        errlog_file = cursor.fetchone()
        
            
        if (errlog_file is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        try:
            log = bz2.decompress(errlog_file[0]).decode('utf-8')
            # fatal error
            m1 = re.findall('.* error:.+', log)
            m2 = re.findall('. undefined reference.+', log)
            # '1:' corresponds to the error status I guess
            m3 = re.findall('.* 1:.+', log)
            # may not be useful: make errors
            m4 = re.findall('make.*: \*\*\*.+', log)
            return m1 + m2 + m3 + m4
        except Exception as e:
            print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
            exit(-1)
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

#cids = [61083, 57231, 34513, 52365, 52941, 62788, 73627, 55836, 70534, 55064]
#cids = [35706, 57116, 33574, 56926, 44782, 78452, 37565, 65481, 77080, 60274]
cids = [31273]
for cid in cids:
    print('errors for ' + str(cid) + ': ' + str(get_errors(cid, False)))
    print('************')
        

In [ ]:
import re
import MySQLdb
import bz2
       
cids = []
try:
    socket = MySQLdb.connect("148.60.11.195", "script", "irmadb2019", "IrmaDB_prod")
    cursor = socket.cursor()
    #query = "SELECT cid FROM Compilations WHERE core_size=-1 and cid > 30000 and cid < 100000"
    query = "SELECT cid FROM Compilations WHERE core_size=-1 and cid > 30000 and cid < 33000"
    #query = "SELECT cid FROM Compilations WHERE cid in ()"
    #cid = 78276 
    cursor.execute(query)
    row = cursor.fetchone()
    while row is not None:
        cids.append(int(str(row).replace("(","").replace(",","").replace(")","")))
        row = cursor.fetchone()
    print('CIDS - ',cids)

    lista = []
    listoflist = []
    tamanho = 0

    for cid in cids:
       
        query = "SELECT errlog_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        errlog_file = cursor.fetchone()
        
        #print(cid)
        
        if (errlog_file is None):
           print ("Unable to retrieve cid=", str(cid))

        try:
           log = bz2.decompress(errlog_file[0]).decode('utf-8')
           # fatal error
           #m1 = re.findall('.* error:.+', log)
           m1 = re.findall('.* error:.+', log)
           #m2 = re.findall('.* undefined reference.+', log)
           m2 = re.findall('. undefined reference to `gen_pool.', log)
           # '1:' corresponds to the error status I guess
           #m3 = re.search('. undefined reference to \'v4l2.+', log)
           #m4 = ('. undefined reference to \'drm.+', log)
           m5 = re.findall('.* 1:.+', log)
           # may not be useful: make errors
           #m4 = re.findall('make.*: \*\*\*.+', log)
           log_error = str(m1) + str(m2) + str(m5)
        except Exception as e:
           print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
           exit(-1)
    
        ja_existe=False
        if tamanho==0:
           lista=[log_error,cid]
           listoflist.append(lista)
           tamanho=1
        else:
            for i in range(tamanho):
                if (listoflist[i][0]==log_error):
                    listoflist[i].append(cid)
                    ja_existe=True
            if ja_existe==False:
                lista=[log_error,cid]
                listoflist.append(lista)
                tamanho=tamanho+1

except Exception as e:
    print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
    exit(-1)    
finally:
    cursor.close()
    socket.close()

file_log = open('clusters.csv', 'w')

for i in range(tamanho):
    ncids=len(listoflist[i])
    #print(listoflist[i][0],';')
    reg = (str(listoflist[i][0])+';')
    for j in range(ncids-1):
        reg = (reg+str(listoflist[i][j+1])+';')
        #print('CIDS: ',listoflist[i][j+1])

    file_log.write(reg+'\n')

file_log.close()

In [ ]:
import pandas as pd
import numpy as np

# download config_bdd30-100.pkl here 
# https://filesender.renater.fr/?s=download&token=556ae671-4546-493e-f96f-3cd3c1919724#

# TODO: excellent "test" set since 15K and 30K have not been subject to specialization
# pd.read_csv('config_bdd15000-20000.csv') 
# pd.read_csv('config_bdd20000-30000.csv') 
rawtuxdata = pd.read_pickle('config_bdd30-100.pkl')

basic_head = ["cid", "time", "date"] # "compile"
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

print("some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course")
# we only keep X86_64 configurations
#rawtuxdata = rawtuxdata[rawtuxdata['X86_64'] == 'y'] ### TODO: I've impression it's not the most effective way (wrt memory) to filter 
if 'X86_64' in rawtuxdata.columns:
    print(rawtuxdata['X86_64'].describe())
    rawtuxdata.query("X86_64 == 'y'", inplace=True) # only keep them

In [ ]:
### TODO: ensure that the same configuration leads to the same compilation status (normally it's the case)
# remove entries with same configurations
print(str(len(rawtuxdata)) + " before the removal of some entries (those with same configurations)")
# tuxdata.drop_duplicates(subset=tuxdata.columns.difference(['vmlinux']), inplace=True)
rawtuxdata.drop_duplicates(subset=rawtuxdata.columns.difference(size_methods).difference(basic_head), inplace=True)
print(str(len(rawtuxdata)) + " after the removal of some entries (those with same configurations)")

In [ ]:
cid_to_remove=74459 # compilation time is very low (2 sec)
cid_to_remove2=30698 # size is 0! 

rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove]
rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove2]

In [ ]:
len(rawtuxdata)

In [ ]:
len(rawtuxdata.query("vmlinux == -1").index)

In [ ]:
def nbyes(row):
    return sum(row == "y")                    
rawtuxdata['nbyes'] = rawtuxdata.apply(nbyes, axis=1)
rawtuxdata.sort_values(by="nbyes")[:10]

In [ ]:
import pandas as pd

errors = pd.read_csv('errors.csv')

rawtuxdata.query("vmlinux == -1", inplace=True) # only keep cids with kernel size == -1 
#print (len(rawtuxdata))

rawtuxdata['cluster'] = 'null'
num = len(rawtuxdata.columns) - 1 

i = 0
for row in rawtuxdata.iterrows():
    cid = rawtuxdata.iloc[i, 0]
    log = errors.query("cid == " + str(cid))['error'].iloc[0]
    rawtuxdata.iloc[i, num] = log
    i = i+1

rawtuxdata[:10]
  
#if (not (errors.query("cid == " + str(cid)).empty)):
    

In [ ]:
errors = pd.read_csv('errors.csv')
clusters = []

#rawtuxdata.query("vmlinux == -1", inplace=True) 
#print (len(rawtuxdata))
#rawtuxdata['cluster'] = 'null'
#num = len(rawtuxdata.columns) - 1 
# query("vmlinux == -1").
i = 0
for row in rawtuxdata.iterrows():
    cid = row[1]['cid']
    if (row[1]['vmlinux'] != -1):
        log = 'NO_FAILURE'
    else:
        cid_error = errors.query("cid == " + str(cid))
        if (len(cid_error) > 0):
            log = cid_error['error'].iloc[0]
        else:
            print("strange case (no cid?)")
    clusters.append(log)
    #rawtuxdata.iloc[i, num] = log
#    i = i + 1
#rawtuxdata[:10]
#pd.merge(rawtuxdata, errors, on='cid', how='outer')

rawtuxdata['cluster_error'] = clusters

In [ ]:
rawtuxdata.query("cluster_error != 'NO_FAILURE'")[['cid', 'cluster_error', 'vmlinux']][:10]

In [ ]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")['cluster_error'].unique()

In [ ]:
#rawtuxdata.cluster_error.count()
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").groupby('cluster_error').count()[['cid']]

In [ ]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y' & cluster_error == 'L24'").groupby(['AIC79XX_BUILD_FIRMWARE', 'WANXL_BUILD_FIRMWARE', 'GENERIC_ALLOCATOR', 'DRM_VBOXVIDEO']).count()[['cid']]

#in 10106 of 1149 errors the option GENERIC_ALLOCATOR is also selected


In [ ]:
rawtuxdata.query("GENERIC_ALLOCATOR == 'y'").groupby('cluster_error').count()[['cid']]

In [ ]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].unique()

In [ ]:
from pandas import DataFrame
import MySQLdb
import bz2

cids = []
errors = []
       
try:
    socket = MySQLdb.connect("148.60.11.195", "script", "irmadb2019", "IrmaDB_prod")
    cursor = socket.cursor()
    cursor.execute("SELECT cid, errlog_file FROM Compilations WHERE core_size = -1 and cid >= 30000 and cid <= 100000")
    #cursor.execute("SELECT cid, errlog_file FROM Compilations WHERE cid in (35706, 57116, 33574, 56926, 44782, 78452, 37565, 65481, 77080, 60274)")
    
    for row in cursor.fetchall():
        cid = row[0]   
        if (row[1] is None):
           print ("Unable to retrieve cid=", str(cid))
        try:
           log_error = bz2.decompress(row[1]).decode('utf-8')
           #log_error = log_error.replace('\r', '').replace('\n', '')
        except Exception as e:
           print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
           exit(-1)
        
        cids.append (cid)
        errors.append (log_error)
    
    Errlogs = {'cid': cids, 'error_message': errors}
    df = DataFrame(Errlogs, columns= ['cid', 'error_message'])
    export_csv = df.to_csv ('errlogs.csv', index = None, header=True) 
    
except Exception as e:
    print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
    exit(-1)    
finally:
    cursor.close()
    socket.close()

In [ ]:
df['cid']

In [ ]:
df.query("cid == '57116'")['error_message']

In [ ]:
len(df)

In [ ]:
df.info(verbose=False)

In [ ]:
df.to_pickle ('errlogs.pkl') # , index = None, header=True) 